# Setup

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import json
import os
from twilio.rest import Client
# https://chrisyeh96.github.io/2017/08/08/definitive-guide-python-imports.html
import config

URL = "https://www.dhhs.vic.gov.au/coronavirus"
OUTPUT_FILE = "./output/output.txt"

# Your Account Sid and Auth Token from twilio.com/console
account_sid = config.TWILIO_ACCOUNT_SID
auth_token = config.TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

today = date.today()
today = today.strftime("%Y-%m-%d")

f1 = open(OUTPUT_FILE, 'w+')

In [2]:
def checkCurrentNumbers():
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find(class_='covid-number-box')

    comment = results.find_all('p')
    comment = comment[0].text
    
    updated_date = soup.find(class_='updated').text
    print(updated_date)

    number = results.find(class_='numbers').text
    
    return comment, number

def check_if_updated():
    if os.path.getsize(OUTPUT_FILE) == 0:
        return False
    data = json.loads(f1.read())
    if today in data.keys() and 'VIC' in data[today].keys():
        print("[DEBUG] Already updated")
        return True
    else:
        print("[DEBUG] Need to update")
        return False

def writeToFile(number):
    if os.path.getsize(OUTPUT_FILE) != 0:
        data = json.loads(f1.read())
    else:
        data = {}
        data[today] = {}
    data[today]["VIC"] = number
    json.dump(data,f1)

def sendSms(comment, number):
    sms_body = f"{comment}\n VIC: {number}"
    client.messages.create(body=sms_body, from_=config.FROM_NUMBER, to=config.TO_NUMBER)

In [3]:
def main():
#     if not check_if_updated():
#         print("")
#         return
#     else:       
        #get current numbers
    comment, number = checkCurrentNumbers()
    #read file and check if current in file
    writeToFile(number)
    print(comment, number)
#     sendSms(comment, number)

In [4]:
main()

Updated: 25 July 2020   04:54pm
new cases (last 24 hours) 357
